In [84]:
import numpy as np
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
stop_words = stopwords.words('english')
import tensorflow as tf
import transformers
from sklearn.decomposition import TruncatedSVD

In [20]:
def read_txt_as_str(f_name):
    with open(f_name, 'r') as file:
        return file.read().replace('\n', ' ')

In [86]:
positive_car = read_txt_as_str("data/positive_car.txt")
negative_car = read_txt_as_str("data/negative_car.txt")

advertising_keywords = ["car", "drive", "fast"]

In [60]:
def preproccess_txt(txt):
    # tokenize
    tokens = word_tokenize(txt.lower())

    # lemmatize
    lemmatizer = WordNetLemmatizer()
    lemmas = [lemmatizer.lemmatize(token) for token in tokens]

    # Remove stopwords
    final_tokens = [token for token in lemmas if token not in stop_words]
    return ' '.join(final_tokens)

In [54]:
def get_tfid_vec(txt):
    vectorizer = TfidfVectorizer(lowercase=True, stop_words='english', strip_accents='ascii')
    vector = vectorizer.fit_transform(txt)
    return vector

In [62]:
def get_lsa(vecs):
    lsa = TruncatedSVD()
    dtm_lsa = lsa.fit_transform(vecs)
    return dtm_lsa

In [45]:
def keyword_2_doc(keywords):
    return " ".join(keywords)

In [82]:
advertising_keywords = ["coffee", "tea", "relax"]

In [87]:
vecs = get_tfid_vec([positive_car, negative_car, keyword_2_doc(advertising_keywords)])
dtm_lsa = get_lsa(vecs)
cosine_similarity(dtm_lsa)

array([[ 1.        ,  0.72557916,  0.51084146],
       [ 0.72557916,  1.        , -0.2209197 ],
       [ 0.51084146, -0.2209197 ,  1.        ]])